### loading modules and data

In [0]:
########################################################

# If running with Google Colab

from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# # If using colab
 from google.colab import files
 files.upload()
# # select the 3 .py files (models, utils, data_utils)

In [0]:
########################################################

# If running with Google Colab
# Create a folder "C247" and then store the project datasets within that folder
# Check that your datasets are setup correctly

!ls "/content/gdrive/My Drive/C247" # File path

In [0]:
from models import *
from utils import *
from data_utils import *

In [0]:
# X_test = np.load("X_test.npy")
# y_test = np.load("y_test.npy")
# person_train_valid = np.load("person_train_valid.npy")
# X_train_valid = np.load("X_train_valid.npy")
# y_train_valid = np.load("y_train_valid.npy")
# person_test = np.load("person_test.npy")

# Change if your directory is different

# dataset_path = './data/' # Yiming Path
dataset_path = "/content/gdrive/My Drive/C247/" 

X_test = np.load(dataset_path + "X_test.npy")
y_test = np.load(dataset_path + "y_test.npy")
person_train_valid = np.load(dataset_path + "person_train_valid.npy")
X_train_valid = np.load(dataset_path + "X_train_valid.npy")
y_train_valid = np.load(dataset_path + "y_train_valid.npy")
person_test = np.load(dataset_path + "person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

#train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)


### K-Fold Training and Validation (use k=1 to get results faster)

Data $\rightarrow$ Window $\rightarrow$ STFT

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 100
window_stride = 40
vote_num = 30 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 8
cwt_scale = 2.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 5

aug_1 = "cwt"
aug_2 = "window"


#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

### Training and Testing

#### Test 1

In [0]:
print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 150
window_stride = 50
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 8
cwt_scale = 2.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "cwt"
aug_2 = "window"


#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

In [0]:
print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

### Test 2

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 150
window_stride = 50
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 8
cwt_scale = 1.0

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 25
hidden_size = 50
lr = 1e-3
weight_decay = 2e-3
num_folds = 1

aug_1 = "cwt"
aug_2 = "window"


#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

In [0]:
print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

### Test 3

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 200
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 9
cwt_scale = 1.0

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 30
hidden_size = 50
lr = 2e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "cwt"
aug_2 = "window"


#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

In [0]:
print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

### Test 4

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 200
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 9
cwt_scale = 1.0

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 30
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "window"
aug_2 = "cwt"


#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

In [0]:
print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)

### Test

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 9
cwt_scale = 0.7

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 30
hidden_size = 50
lr = 2e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "cwt"
aug_2 = "window"


#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

In [0]:
print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

### Testtttt

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 9
cwt_scale = 0.4

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 30
hidden_size = 50
lr = 2e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "cwt"
aug_2 = "window"


#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

## Bandpass Filter Function

In [0]:
def bandpass_filter(X,low,high):
  N, C, T = X.shape
  out = np.zeros_like(X)
  nyq = 125 #nyquist frequency, highest able to be sensed for this data
  if high > nyq :
    high = nyq
  order = 9

  b, a = sig.butter(order, [low/nyq,high/nyq], btype='band')
  out = sig.lfilter(b, a, X, axis=-1)

  return out

## Tests with bandpass

### Test

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 200
window_stride = 50
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 9
cwt_scale = 1.0

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 30
hidden_size = 50
lr = 2e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "cwt"
aug_2 = "window"

#BANDPASS PARAMS
band_low = 7
band_high = 30

#bandpass the data
X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

### Test Again

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 10
cwt_scale = 2

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 40
hidden_size = 50
lr = 2e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "cwt"
aug_2 = "window"

#BANDPASS PARAMS
band_low = 5
band_high = 35

#bandpass the data
X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

### Using weird windowing

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 200
window_stride = 50
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 10
cwt_scale = 0.4

#RNN PARAMS
rnn_type = 'CNNGRUnet'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 2e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = "cwt2"
aug_2 = "window"

#BANDPASS PARAMS
band_low = 7
band_high = 30

#bandpass the data
X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

### Repeating Josh's test

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 200
window_stride = 50
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 100
stft_stride = 5

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.6

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 0
num_folds = 1

aug_1 = "cwt2"
aug_2 = "window"

#BANDPASS PARAMS
band_low = 7
band_high = 30

#bandpass the data
X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

### Larger Window

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 125
stft_stride = 5

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 0
num_folds = 1

aug_1 = "window"
aug_2 = "stft"

#BANDPASS PARAMS
band_low = 7
band_high = 30

#bandpass the data
X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


for k in range(num_folds):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
    X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
    print('Initial fold shape: {}'.format(X_train.shape))
    #apply second augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
    
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('{} applied...'.format(aug_2))
    print('Data shape: {}'.format(X_train.shape))
    
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

print('Augmenting test data...')
X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
#X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_2,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('Testing...')
TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale, vote_num=vote_num)

### STFT Grid Search

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_sizes = range(10,200,10)
stft_strides = range(1,10)

test_acc = np.empty((len(stft_sizes),len(stft_strides)))

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = None
aug_2 = "stft"

#BANDPASS PARAMS
#band_low = 7
#band_high = 30

#bandpass the data
#X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1)#,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)
for i in range(len(stft_sizes)):
  for j in range(len(stft_strides)):
    stft_size = stft_sizes[i]
    stft_stride = stft_strides[j]
    print('CURRENT STFT SIZE: %s CURRENT STFT STRIDE: %s'%(stft_size,stft_stride))
    for k in range(num_folds):
        # indicate hyperparameters here
        print ('fold {}'.format(k+1))
        X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
        X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
        print('Initial fold shape: {}'.format(X_train.shape))
        #apply second augmentation
        X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
        
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
        print('{} applied...'.format(aug_2))
        print('Data shape: {}'.format(X_train.shape))
        
        model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
        EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
        EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
        EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
        EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
        print('Data prepared, model initialized, beginning training...')
        best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

    print('Augmenting test data...')
    X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    #X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_2,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    print('Testing...')
    test_acc[i,j] = TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size,vote_num=vote_num,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_sizes = range(40,200,10)
stft_strides = range(1,10)

test_acc = np.empty((len(stft_sizes),len(stft_strides)))

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = None
aug_2 = "stft"

#BANDPASS PARAMS
#band_low = 7
#band_high = 30

#bandpass the data
#X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1)#,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)
for i in range(len(stft_sizes)):
  for j in range(len(stft_strides)):
    stft_size = stft_sizes[i]
    stft_stride = stft_strides[j]
    print('CURRENT STFT SIZE: %s CURRENT STFT STRIDE: %s'%(stft_size,stft_stride))
    for k in range(num_folds):
        # indicate hyperparameters here
        print ('fold {}'.format(k+1))
        X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
        X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
        print('Initial fold shape: {}'.format(X_train.shape))
        #apply second augmentation
        X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
        
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
        print('{} applied...'.format(aug_2))
        print('Data shape: {}'.format(X_train.shape))
        
        model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
        EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
        EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
        EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
        EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
        print('Data prepared, model initialized, beginning training...')
        best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

    print('Augmenting test data...')
    X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    #X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_2,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    print('Testing...')
    test_acc[i,j] = TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size,vote_num=vote_num,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_sizes = range(100,200,10)
stft_strides = range(4,10,2)

test_acc = np.empty((len(stft_sizes),len(stft_strides)))

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = None
aug_2 = "stft"

#BANDPASS PARAMS
#band_low = 7
#band_high = 30

#bandpass the data
#X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1)#,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)
for i in range(len(stft_sizes)):
  for j in range(len(stft_strides)):
    stft_size = stft_sizes[i]
    stft_stride = stft_strides[j]
    print('CURRENT STFT SIZE: %s CURRENT STFT STRIDE: %s'%(stft_size,stft_stride))
    for k in range(num_folds):
        # indicate hyperparameters here
        print ('fold {}'.format(k+1))
        X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
        X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
        print('Initial fold shape: {}'.format(X_train.shape))
        #apply second augmentation
        X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
        
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
        print('{} applied...'.format(aug_2))
        print('Data shape: {}'.format(X_train.shape))
        
        model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
        EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
        EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
        EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
        EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
        print('Data prepared, model initialized, beginning training...')
        best_model = TrainValRNN(model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

    print('Augmenting test data...')
    X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    #X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_2,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    print('Testing...')
    test_acc[i,j] = TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size,vote_num=vote_num,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_sizes = range(100,1000,50)
stft_strides = range(5,20,5)

test_acc = np.empty((len(stft_sizes),len(stft_strides)))

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = None
aug_2 = "stft"

#BANDPASS PARAMS
band_low = 7
band_high = 30

#bandpass the data
X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1)#,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)
for i in range(len(stft_sizes)):
  for j in range(len(stft_strides)):
    stft_size = stft_sizes[i]
    stft_stride = stft_strides[j]
    print('CURRENT STFT SIZE: %s CURRENT STFT STRIDE: %s'%(stft_size,stft_stride))
    for k in range(num_folds):
        # indicate hyperparameters here
        print ('fold {}'.format(k+1))
        X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
        X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
        print('Initial fold shape: {}'.format(X_train.shape))
        #apply second augmentation
        X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
        
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
        print('{} applied...'.format(aug_2))
        print('Data shape: {}'.format(X_train.shape))
        
        model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
        EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
        EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
        EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
        EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
        print('Data prepared, model initialized, beginning training...')
        best_model = TrainValRNN(verbose=False,model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

    print('Augmenting test data...')
    X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    #X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_2,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    print('Testing...')
    test_acc[i,j] = TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size,vote_num=vote_num,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 250
window_stride = 75
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_sizes = range(300,1000,100)
stft_strides = range(5,20,5)

test_acc = np.empty((len(stft_sizes),len(stft_strides)))

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = None
aug_2 = "stft"

#BANDPASS PARAMS
band_low = 7
band_high = 30

#bandpass the data
X_train_valid = bandpass_filter(X_train_valid, band_low, band_high)
print('Data bandpass filtered...')

#make data N(0,1)
mu = np.mean(X_train_valid, axis=(0,2), keepdims = True) 
std = np.std(X_train_valid, axis=(0,2), keepdims = True)
X_train_valid = standardize(X_train_valid, mu, std)
print('Data normalized...')
print('Data shape: {}'.format(X_train_valid.shape))

# apply first augmentation
X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid,y_train_valid,person_train_valid,aug_type=aug_1)#,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
print('{} applied...'.format(aug_1))
train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)
for i in range(len(stft_sizes)):
  for j in range(len(stft_strides)):
    stft_size = stft_sizes[i]
    stft_stride = stft_strides[j]
    print('CURRENT STFT SIZE: %s CURRENT STFT STRIDE: %s'%(stft_size,stft_stride))
    for k in range(num_folds):
        # indicate hyperparameters here
        print ('fold {}'.format(k+1))
        X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
        X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
        print('Initial fold shape: {}'.format(X_train.shape))
        #apply second augmentation
        X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
        
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
        print('{} applied...'.format(aug_2))
        print('Data shape: {}'.format(X_train.shape))
        
        model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
        EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
        EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
        EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
        EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
        print('Data prepared, model initialized, beginning training...')
        best_model = TrainValRNN(verbose=False,model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

    print('Augmenting test data...')
    X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    #X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_2,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    print('Testing...')
    test_acc[i,j] = TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size,vote_num=vote_num,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    

## Bandpass + Window grid search on bandpass

In [0]:
# setting up the data augmentation here
#aug_type = 'cwt'

#WINDOW PARAMS
window_size = 200
window_stride = 20
vote_num = 50 # how many samples should vote to classify?

#STFT PARAMS
stft_size = 125
stft_stride = 5

#CWT PARAMS
cwt_level = 8
cwt_scale = 0.5

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3
num_folds = 1

aug_1 = None
aug_2 = "window"

lows = range(2,16,2)
highs = range(20,50,5)


for low in lows:
  for high in highs:
    #BANDPASS PARAMS
    band_low = low
    band_high = high

    #bandpass the data
    X_train_valid_filt = bandpass_filter(X_train_valid, band_low, band_high)
    print('Data bandpass filtered with low = %s and high = %s ...'%(low,high))

    #make data N(0,1)
    mu = np.mean(X_train_valid_filt, axis=(0,2), keepdims = True) 
    std = np.std(X_train_valid_filt, axis=(0,2), keepdims = True)
    X_train_valid = standardize(X_train_valid_filt, mu, std)
    print('Data normalized...')
    print('Data shape: {}'.format(X_train_valid.shape))

    # apply first augmentation
    X_train_valid_aug,y_train_valid_aug,person_train_valid_aug = Aug_Data(X_train_valid_filt,y_train_valid,person_train_valid,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    print('{} applied...'.format(aug_1))
    train_fold, val_fold = Train_Val_Data(X_train_valid_aug, y_train_valid_aug)


    for k in range(num_folds):
        # indicate hyperparameters here
        print ('fold {}'.format(k+1))
        X_train, y_train, p_train = X_train_valid_aug[train_fold[k]], y_train_valid_aug[train_fold[k]], person_train_valid_aug[train_fold[k]]
        X_val, y_val, p_val = X_train_valid_aug[val_fold[k]], y_train_valid_aug[val_fold[k]], person_train_valid_aug[val_fold[k]]
        print('Initial fold shape: {}'.format(X_train.shape))
        #apply second augmentation
        X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_2, cwt_level=cwt_level, cwt_scale=cwt_scale, stft_size=stft_size, stft_stride=stft_stride, window_size=window_size, window_stride=window_stride)
        
        #X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_2, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
        print('{} applied...'.format(aug_2))
        print('Data shape: {}'.format(X_train.shape))
        
        model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1], hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
        EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
        EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
        EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
        EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
        print('Data prepared, model initialized, beginning training...')
        best_model = TrainValRNN(verbose=False,model=model, criterion=criterion, optimizer=optimizer, trainloader=EEG_trainloader, valloader=EEG_valloader, num_epochs=num_epochs, aug_type=aug_2, window_size=window_size, vote_num=vote_num)

    print('Augmenting test data...')
    X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_1,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    #X_test_aug, y_test_aug, person_test_aug = Aug_Data(X_test,y_test,person_test,aug_type=aug_2,window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale)
    print('Testing...')
    TestRNN(best_model, X_test_aug, y_test_aug, person_test_aug, aug_type=aug_2, window_size=window_size,window_stride=window_stride,stft_size=stft_size,stft_stride=stft_stride,cwt_level=cwt_level,cwt_scale=cwt_scale, vote_num=vote_num)